# مدل سازی بدون استفاده از ویژگی ها اضافی

## وارد کردن کتابخانه ها

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
import warnings

warnings.filterwarnings('ignore')

## استارت h2o

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 17.0.6+9-LTS-190, mixed mode, sharing)
  Starting server from D:\Projects\LifeExpectancy\venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\amira\AppData\Local\Temp\tmpq068mi0r
  JVM stdout: C:\Users\amira\AppData\Local\Temp\tmpq068mi0r\h2o_amira_started_from_python.out
  JVM stderr: C:\Users\amira\AppData\Local\Temp\tmpq068mi0r\h2o_amira_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Asia/Tehran
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,9 days
H2O_cluster_name:,H2O_from_python_amira_9goljf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.975 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


## خواندن دیتا

In [3]:
data = h2o.import_file('../../data/ML_Data/Data_2031.csv')

data = data.drop(['ISO3 Alpha-code', 'GDP',
                  'Educated_Population', 'Suicide', 'HIV', 'BMI', 'darmani*',
                  'Estimated road traffic death rate (per 100 000 population)',
                  'Hepatitis B (HepB3) immunization coverage among 1-year-olds (%)'])
data

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Country,subregion,region,Year,Births (thousands),"Population Density, as of 1 July (persons per square km)","Population Sex Ratio, as of 1 July (males per 100 females)",Population Growth Rate (percentage),"Rate of Natural Change (per 1,000 population)","Crude Birth Rate (births per 1,000 population)",Mean Age Childbearing (years),Sex Ratio at Birth (males per 100 female births),"Crude Death Rate (deaths per 1,000 population)","Under-Five Deaths, under age 5 (thousands)",Net Number of Migrants (thousands),"Net Migration Rate (per 1,000 population)","Life Expectancy at Birth, both sexes (years)"
Burundi,Eastern Africa,AFRICA,1950,116.224,86.864,91.947,2.2,27.788,51.334,30.995,102.5,23.546,29.001,-13.343,-5.893,40.579
Burundi,Eastern Africa,AFRICA,1951,117.067,88.757,92.145,2.114,26.799,50.678,30.996,102.5,23.879,30.669,-13.217,-5.721,40.812
Burundi,Eastern Africa,AFRICA,1952,117.77,90.618,92.319,2.036,26.118,49.933,31.026,102.5,23.815,31.272,-13.715,-5.815,41.003
Burundi,Eastern Africa,AFRICA,1953,119.013,92.451,92.488,1.969,25.844,49.448,31.03,102.5,23.604,31.644,-14.962,-6.216,41.282
Burundi,Eastern Africa,AFRICA,1954,119.972,94.287,92.65,1.965,25.536,48.883,31.036,102.5,23.347,31.915,-14.599,-5.948,41.558
Burundi,Eastern Africa,AFRICA,1955,121.13,96.154,92.806,1.956,25.36,48.399,31.063,102.5,23.039,31.984,-14.669,-5.861,41.855
Burundi,Eastern Africa,AFRICA,1956,122.196,98.045,92.956,1.939,25.101,47.885,31.073,102.5,22.784,31.908,-14.733,-5.773,42.059
Burundi,Eastern Africa,AFRICA,1957,123.309,99.955,93.106,1.921,24.93,47.398,31.086,102.5,22.468,31.831,-15.021,-5.774,42.329
Burundi,Eastern Africa,AFRICA,1958,124.674,101.856,93.25,1.846,24.828,47.013,31.125,102.5,22.185,31.769,-17.047,-6.428,42.559
Burundi,Eastern Africa,AFRICA,1959,126.251,103.801,93.392,1.937,24.801,46.736,31.151,102.5,21.935,31.804,-14.813,-5.483,42.779


## جدا سازی داده های تست و ولیدیشن و آموزش

In [4]:
test = data[data["Year"] > 2021]
train_val_data = data[data["Year"] <= 2021]

x = train_val_data.columns
y = 'Life Expectancy at Birth, both sexes (years)'
x.remove(y)

ss = data.split_frame(seed=1)
train = ss[0]
valid = ss[1]

## مشخص کردن هایپر پارامترها

In [5]:
drf_params1 = {'ntrees': [50, 75, 100],
             'max_depth': [5, 10, 15],
             'min_rows': [1, 5]}

drf_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          grid_id='drf_grid1',
                          hyper_params=drf_params1)

## آموزش مدل

In [6]:
drf_grid1.train(x=x, y=y,
                training_frame=train,
                validation_frame=valid,
                seed=1)

gbm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


,max_depth,min_rows,ntrees,model_ids,residual_deviance
,10.0,1.0,100.0,drf_grid1_model_14,0.1853386
,10.0,5.0,100.0,drf_grid1_model_17,0.1872164
,10.0,1.0,75.0,drf_grid1_model_8,0.1907811
,10.0,5.0,75.0,drf_grid1_model_11,0.1955388
,10.0,1.0,50.0,drf_grid1_model_2,0.2087547
,10.0,5.0,50.0,drf_grid1_model_5,0.2206367
,15.0,5.0,100.0,drf_grid1_model_18,0.2445711
,15.0,5.0,75.0,drf_grid1_model_12,0.2459740
,15.0,5.0,50.0,drf_grid1_model_6,0.2654462
,15.0,1.0,100.0,drf_grid1_model_15,0.3044053


## مرتب کردن مدل ها بر اساس R^2

In [7]:
drf_gridperf1 = drf_grid1.get_grid(sort_by='r2', decreasing=True)
drf_gridperf1

,max_depth,min_rows,ntrees,model_ids,r2
,10.0,1.0,100.0,drf_grid1_model_14,0.9986234
,10.0,5.0,100.0,drf_grid1_model_17,0.9986095
,10.0,1.0,75.0,drf_grid1_model_8,0.9985830
,10.0,5.0,75.0,drf_grid1_model_11,0.9985477
,10.0,1.0,50.0,drf_grid1_model_2,0.9984495
,10.0,5.0,50.0,drf_grid1_model_5,0.9983612
,15.0,5.0,100.0,drf_grid1_model_18,0.9981835
,15.0,5.0,75.0,drf_grid1_model_12,0.9981731
,15.0,5.0,50.0,drf_grid1_model_6,0.9980284
,15.0,1.0,100.0,drf_grid1_model_15,0.9977391


## انتخاب بهترین مدل

In [8]:
best_drf1 = drf_gridperf1.models[0]

In [15]:
best_drf1.train(x=x, y=y, training_frame=train, validation_frame=valid)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,50.0,50.0,43992.0,5.0,5.0,5.0,23.0,32.0,30.16
,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2023-02-18 18:06:10,0.012 sec,0.0,11.5340839,9.3315599,133.0350912,11.6033521,9.3670447,134.6377798
,2023-02-18 18:06:10,0.087 sec,1.0,10.4336925,8.4400169,108.8619396,10.5002133,8.4809017,110.2544803
,2023-02-18 18:06:10,0.141 sec,2.0,9.4462491,7.6453985,89.2316227,9.5125253,7.6906379,90.4881384
,2023-02-18 18:06:10,0.171 sec,3.0,8.5501861,6.9205831,73.1056830,8.6126872,6.9649826,74.1783800
,2023-02-18 18:06:10,0.208 sec,4.0,7.7398614,6.2663843,59.9054551,7.7998186,6.3100385,60.8371701
,2023-02-18 18:06:10,0.239 sec,5.0,7.0081909,5.6772196,49.1147395,7.0623508,5.7187869,49.8767989
,2023-02-18 18:06:10,0.264 sec,6.0,6.3521397,5.1460720,40.3496785,6.4073396,5.1879824,41.0540002
,2023-02-18 18:06:10,0.302 sec,7.0,5.7649659,4.6720757,33.2348315,5.8179054,4.7109053,33.8480228


In [16]:
best_drf1.r2()

0.9969081012329075

## ارزیابی مدل

In [17]:
best_drf_perf1 = best_drf1.model_performance(test)

best_drf_perf1.r2()

0.9941126964710356

In [13]:
pred_test = best_drf1.predict(test)
pred_test

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict
61.9444
62.5123
63.1949
63.42
63.6265
63.7489
64.0071
64.2489
64.2809
64.458
